In [27]:
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

#import os
#pathName = '/home/ubuntu/data/PEC/model_dict/ml100k/'
#if not os.path.exists(pathName):
#     os.makedirs(pathName)
#MODELPATH = pathName + 'dnn_v1.0_'

data_path = '/home/zhangxz/workspace/dataset/ml-100k/'
UIT = pd.read_csv(data_path + 'ml_header.trace')
simUsers = np.loadtxt(data_path+"simUser.csv",  delimiter=',',dtype=int)

trainUIT = UIT[UIT['day']<max(UIT['day']+1)*0.6]
validUIT = UIT[UIT['day']<max(UIT['day']+1)*0.8]

contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())

rewardPara = {"alpha":0.01,"betao":1,"betal":1}
latency = [0.2,1,0.8]
Bu = 20

BATCH_SIZE = 128
GAMMA = 0.99

num_episodes = 10
EPS_START = 0.99
EPS_END = 0.01
EPS_DECAY = trainUIT.shape[0]*3
agentStep = 0

torch.cuda.empty_cache()
UIT.shape,trainUIT.shape,validUIT.shape

((132690, 4), (76252, 4), (106994, 4))

In [28]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        #self.embedding = nn.Embedding(input_dim, emb_dim)
        
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        #embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        embedded = self.dropout(src)
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell
    
    
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        #self.embedding = nn.Embedding(output_dim, emb_dim)
        
        #self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.rnn = nn.LSTM(output_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input_ = input_.float().unsqueeze(0)
        #print("decode input shape",input_.shape)
        #input = [1, batch size]
        
        #embedded = self.dropout(self.embedding(input))
        embedded = self.dropout(input_)
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        #prediction = self.fc_out(output.squeeze(0)).unsqueeze(-1)
        prediction = self.fc_out(output.squeeze(0))
        #print(prediction.shape)
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        #print("trg_len",trg.shape[0],"batch_size",trg.shape[1])
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        #print(input.shape)
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            #top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            #print(trg[t].shape,output.unsqueeze(-1).shape)
            input = trg[t] if teacher_force else output
        
        return outputs

In [30]:
INPUT_DIM = 1
OUTPUT_DIM = 1
ENC_EMB_DIM = 0
DEC_EMB_DIM = 0
HID_DIM = 64
#HID_DIM = 64

N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [31]:
FOLD = 2
CITY = 0
BATCHSIZE = contentNum

model.load_state_dict(torch.load("fold{}-city{}-model.pt".format(FOLD,CITY)))      

<All keys matched successfully>

In [36]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
        #self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit,dpc,data):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1
        
            if data != None:
                actionIndex = self.evaluate(dpc,data,self.Bu+1)
            else:
                actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
            if self.W[-1] not in actionIndex:
                actionIndex.pop() 
            for index in actionIndex:
                action[index] = 1

            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action
    def evaluate(self,model, batch, cachesize):

        model.eval()

        with torch.no_grad():

            src = batch.permute(1,0,2).float().to(device)

            batchsize = src.shape[1]
            trg = torch.zeros((26,batchsize,1)).to(device)

            torch.cuda.empty_cache()

            output = model(src, trg, 0) #turn off teacher forcing

            output = output.squeeze(-1)

            output = output[0].cpu().numpy()

            max_item=np.argsort(-output)[0:cachesize]
            #print(max_item,list(max_item))
        return list(max_item)

In [37]:
UIT

,u,i,day,time
0,46,8033,0,2239
1,363,13244,1,3907
2,363,15904,1,4013
3,979,3842,1,5523
4,979,8313,1,5536
...,...,...,...,...
132685,717,8224,4319,15549531
132686,717,11752,4319,15549743
132687,717,8805,4319,15549756
132688,717,5436,4319,15549819


In [38]:
UIT['day'] = UIT['time']//(60*60)
UIT

,u,i,day,time
0,46,8033,0,2239
1,363,13244,1,3907
2,363,15904,1,4013
3,979,3842,1,5523
4,979,8313,1,5536
...,...,...,...,...
132685,717,8224,4319,15549531
132686,717,11752,4319,15549743
132687,717,8805,4319,15549756
132688,717,5436,4319,15549819


In [39]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

Frq_hour = torch.zeros((BATCHSIZE,720*6)).to(device)
m = 20


rewardPara = {"alpha":1,"betao":1,"betal":1}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)
i_episode = 0

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    if uit[1] in np.argwhere(ue.lastAction):
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    if uit[2]>=m:
        test_iterator = Frq_hour[:,uit[2]-m:uit[2]]/Frq_hour[:,uit[2]-m:uit[2]].sum(dim=0)
        test_iterator = test_iterator.unsqueeze(2)
        #print(test_iterator.sum())
        
    else:
        test_iterator = None
    
    ue.selectAction(env,uit,model,test_iterator)

    for content in np.argwhere(ue.lastAction):
        Frq_hour[content,uit[2]]+=1
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sat Oct 16 23:57:40 2021   Index: 9999   Loss: 0.0 --
Reward: [-5.57074  0.222    0.     ] total reward: -5.34874
UEHitrate: 0.0  edgeHitrate 0.2775 sumHitrate 0.2775  privacy: 1.23117

--Time: Sun Oct 17 00:06:33 2021   Index: 19999   Loss: 0.0 --
Reward: [-5.18181  0.2046   0.     ] total reward: -4.97721
UEHitrate: 0.0  edgeHitrate 0.25575 sumHitrate 0.25575  privacy: 1.07531

--Time: Sun Oct 17 00:15:45 2021   Index: 29999   Loss: 0.0 --
Reward: [-4.86024  0.21331  0.     ] total reward: -4.64693
UEHitrate: 0.0  edgeHitrate 0.26663 sumHitrate 0.26663  privacy: 1.01628

--Time: Sun Oct 17 00:26:08 2021   Index: 39999   Loss: 0.0 --
Reward: [-4.68612  0.20518  0.     ] total reward: -4.48094
UEHitrate: 0.0  edgeHitrate 0.25648 sumHitrate 0.25648  privacy: 0.98865

--Time: Sun Oct 17 00:36:27 2021   Index: 49999   Loss: 0.0 --
Reward: [-4.47599  0.21037  0.     ] total reward: -4.26563
UEHitrate: 0.0  edgeHitrate 0.26296 sumHitrate 0.26296  privacy: 0.96617

--Time: Sun Oct 17

In [40]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.2775 , 0.25575, 0.26663, 0.25648, 0.26296, 0.25502, 0.2484 ,
        0.24264, 0.24707, 0.2435 , 0.25011, 0.25008, 0.24438, 0.24478,
        0.     ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.2775 , 0.25575, 0.26663, 0.25648, 0.26296, 0.25502, 0.2484 ,
        0.24264, 0.24707, 0.2435 , 0.25011, 0.25007, 0.24438, 0.24478,
        0.     ]))

In [41]:
privacyReduction

array([1.23117, 1.07531, 1.01628, 0.98865, 0.96617, 0.95102, 0.94363,
       0.93239, 0.91877, 0.90363, 0.88124, 0.85571, 0.81495, 0.77372,
       0.     ])

In [43]:
class ENV(object):
    def __init__(self,userNum,contentNum,latency,Bu):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum,dtype=np.float32)
        self.e = np.zeros(shape=contentNum,dtype=int)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l = np.array(latency,dtype=np.float32)
        

        self.B = np.full(shape=userNum,fill_value=Bu,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)
        self.r[u] =  self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (self.r,
                self.p, 
                self.e,
                self.S,
                self.l)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()

class UE(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = np.zeros(contentNum,dtype=int)

        self.Bu = Bu
        self.contentNum = contentNum
        self.userNum = userNum

        r , p , e, S, l = env.getStatus()

        #self.lastAction = torch.zeros(size=(contentNum,),dtype=int)
        self.lastAction = np.zeros(contentNum,dtype=int)

        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']             
        self.reward = 0

        self.simU = simUsers[u]
        self.lastStatusFeature = torch.from_numpy(self.statusEmbedding(r,p,e))
        #self.lastp = p.clone()
        #self.lastp = p.copy()
        
        self.lastp = p
        #self.lastr = r.clone()
        #self.lastr = r.copy()
        self.lastru = r[self.u]
        
    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    def statusEmbedding(self,r,p,e):
        tmp = np.zeros(contentNum,dtype=np.float32)
        for simUser in self.simU:
            tmp += r[simUser]
        simUserRu = (tmp / len(self.simU))
        ru = r[self.u]
        statusFeature = np.row_stack((self.v,ru,simUserRu,p,e))
        
        return statusFeature.T.astype(np.float32)
    
    def getReward(self,lastru,lastp,ru,p,i,lastAction,S,l,e,v):
        
        #self.Rh =   self.ALPHAh * (np.log(v * p + (1-v) * (1-p)).sum() / np.log(ru * p + (1-ru) * (1-p)).sum() )

        #self.Rh = self.ALPHAh * ( 1 / ( 1 + np.exp( 0.5 * np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() ) ) )

        self.Rh = - self.ALPHAh * ( np.log( ( lastru * lastp + ( 1 - lastru ) * ( 1 - lastp ) ) / ( ru * p + ( 1 - ru ) * ( 1 - p ) ) ).sum() )


        self.Ro =   self.BETAo * lastAction[i] * S[i] * ( e[i] * l[0] + ( 1 - e[i] ) * l[1] )
        
        # self.Ro =   self.BETAo * lastAction[i] * S[i] * ( 1 + e[i] * l[0] + ( 1 - e[i] ) * l[1] )

        self.Rl =   self.BETAl * ( 1 - lastAction[i] ) * S[i] * e[i] * l[2]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        #return  self.Rh.sum()
        return  (self.Rh + self.Ro + self.Rl).astype(np.float32)

    def selectAction(self,env,uit,dpc,data):
        
        self.updateViewContent(uit[1])
        
        r , p , e, S, l = env.getStatus()
        
        self.reward = self.getReward(self.lastru,self.lastp,r[self.u],p,self.W[-1],self.lastAction,S,l,e,self.v)

        self.lastp = p
        self.lastru = copy.copy(r[self.u])
        
        #self.lastp = p.clone()
        #self.lastr = r.clone()
        
        if self.W[-1] not in np.argwhere(self.lastAction): #local cache miss
            action = np.zeros(contentNum,dtype=int)
            action[self.W[-1]] = 1
        
            if data != None:
                actionIndex = self.evaluate(dpc,data,self.Bu+1)
            else:
                actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
            if self.W[-1] not in actionIndex:
                actionIndex.pop() 
            for index in actionIndex:
                action[index] = 1

            env.updateEnv(self.u,action,uit[2])
        else:
            action = self.lastAction # keep the cache and no request the new video
        return action
    def evaluate(self,model, batch, cachesize):

        model.eval()

        with torch.no_grad():

            src = batch.permute(1,0,2).float().to(device)

            batchsize = src.shape[1]
            trg = torch.zeros((26,batchsize,1)).to(device)

            torch.cuda.empty_cache()

            output = model(src, trg, 0) #turn off teacher forcing

            output = output.squeeze(-1)

            output = output[0].cpu().numpy()

            max_item=np.argsort(-output)[0:cachesize]
            #print(max_item,list(max_item))
        return list(max_item)

In [44]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum,latency,Bu)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

Frq_hour = torch.zeros((BATCHSIZE,720*6)).to(device)
m = 20


rewardPara = {"alpha":1,"betao":1,"betal":1}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)
i_episode = 0

for index,trace in UIT.iterrows():
    uit = trace.to_numpy()
    if uit[0] not in UEs:
        UEs[uit[0]] = UE(uit[0],env,rewardPara)
    ue = UEs[uit[0]]
    
    if uit[1] in np.argwhere(ue.lastAction):
        UEHit[uit[0]] += 1
    elif uit[1] in env.pipe.keys():
        edgeHit += 1
    if uit[2]>=m:
        test_iterator = Frq_hour[:,uit[2]-m:uit[2]]/Frq_hour[:,uit[2]-m:uit[2]].sum(dim=0)
        test_iterator = test_iterator.unsqueeze(2)
        #print(test_iterator.sum())
        
    else:
        test_iterator = None
    
    ue.selectAction(env,uit,model,test_iterator)

    for content in np.argwhere(ue.lastAction):
        Frq_hour[content,uit[2]]+=1
    
    sumReward[0] += float(ue.Rh)
    sumReward[1] += float(ue.Rl)
    sumReward[2] += float(ue.Ro)
    if (index+1) % 10000 == 0 :
        psi = 0
        for u in UEs:
            psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
        print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
        print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
        print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
        print()
        sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
        UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
        edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
        privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
        
psi = 0
for u in UEs:
    psi += np.log(env.r[u] * env.p + (1-env.r[u]) * (1-env.p)).sum() / np.log(UEs[u].v * env.p + (1-UEs[u].v) * (1-env.p)).sum()
print()
print("----------------------------------------------------------------")
print("--Time:",time.asctime( time.localtime(time.time())),"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
print("----------------------------------------------------------------")
print()
sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Oct 17 10:13:50 2021   Index: 9999   Loss: 0.0 --
Reward: [-13.59472   0.22424   0.     ] total reward: -13.37048
UEHitrate: 0.0  edgeHitrate 0.2803 sumHitrate 0.2803  privacy: 2.06928

--Time: Sun Oct 17 10:23:04 2021   Index: 19999   Loss: 0.0 --
Reward: [-9.14193  0.20572  0.     ] total reward: -8.93621
UEHitrate: 0.0  edgeHitrate 0.25715 sumHitrate 0.25715  privacy: 1.65084

--Time: Sun Oct 17 10:32:20 2021   Index: 29999   Loss: 0.0 --
Reward: [-7.46918  0.21405  0.     ] total reward: -7.25512
UEHitrate: 0.0  edgeHitrate 0.26757 sumHitrate 0.26757  privacy: 1.47097

--Time: Sun Oct 17 10:41:58 2021   Index: 39999   Loss: 0.0 --
Reward: [-6.62011  0.20574  0.     ] total reward: -6.41437
UEHitrate: 0.0  edgeHitrate 0.25717 sumHitrate 0.25718  privacy: 1.37198

--Time: Sun Oct 17 10:51:45 2021   Index: 49999   Loss: 0.0 --
Reward: [-6.01392  0.21082  0.     ] total reward: -5.80311
UEHitrate: 0.0  edgeHitrate 0.26352 sumHitrate 0.26352  privacy: 1.29012

--Time: Sun Oc

In [45]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.2803 , 0.25715, 0.26757, 0.25718, 0.26352, 0.25548, 0.2488 ,
        0.24299, 0.24738, 0.24378, 0.25036, 0.25031, 0.2446 , 0.24499,
        0.     ]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.2803 , 0.25715, 0.26757, 0.25717, 0.26352, 0.25548, 0.2488 ,
        0.24299, 0.24738, 0.24378, 0.25036, 0.25031, 0.2446 , 0.24499,
        0.     ]))

In [46]:
privacyReduction

array([2.06928, 1.65084, 1.47097, 1.37198, 1.29012, 1.23405, 1.19906,
       1.15923, 1.12433, 1.08759, 1.04539, 1.00383, 0.94799, 0.89549,
       0.     ])